In [1]:
!pip install opencv-python tensorflow

In [2]:
import cv2
import numpy as np
import tensorflow as tf
import os
from google.colab import files

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
model_path = "/content/drive/MyDrive/classifier/cnnmodel.h5"

In [5]:
model = tf.keras.models.load_model(model_path)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [6]:
import cv2
import numpy as np
import tensorflow as tf

# --- your image path ---
img_path = "/content/drive/MyDrive/classifier/A0767(0.7011).png"

# --- load image and basic check ---
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError(f"Image not found: {img_path}")

# --- inspect model input shape ---
# model should already be loaded in your session
try:
    in_shape = model.input_shape  # e.g. (None, H, W, C)
except Exception:
    # fallback
    in_shape = model.layers[0].input_shape

if len(in_shape) != 4:
    raise ValueError(f"Unsupported model input shape: {in_shape}")

_, H, W, C = in_shape
H = int(H) if in_shape[1] is not None else 224
W = int(W) if in_shape[2] is not None else 224
C = int(C)

print(f"Model expects: {H}x{W}x{C}")

# --- preprocess according to channels ---
# If model expects grayscale (C==1) convert to gray then expand channel dim
if C == 1:
    # convert BGR -> GRAY
    img_proc = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_proc = cv2.resize(img_proc, (W, H))
    # add channel axis: (H,W) -> (H,W,1)
    img_proc = np.expand_dims(img_proc, axis=-1)
elif C == 3:
    # convert BGR -> RGB and resize
    img_proc = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_proc = cv2.resize(img_proc, (W, H))
else:
    raise ValueError(f"Model expects {C} channels — this code only handles 1 or 3.")

# --- normalize and batchify ---
x = img_proc.astype("float32") / 255.0
x = np.expand_dims(x, axis=0)   # (1, H, W, C)

# --- predict ---
pred = model.predict(x)

# handle sigmoid or softmax outputs robustly
pred = np.array(pred)
if pred.ndim == 2 and pred.shape[1] == 1:
    prob = float(pred[0,0])
elif pred.ndim == 2 and pred.shape[1] >= 2:
    prob = float(pred[0,1])   # class-1 prob
else:
    prob = float(np.ravel(pred)[0])

prob = float(prob)  # just making sure
state = ""

if prob < 0.4:
    state = "alert"
elif prob < 0.7:
    state = "slightly drowsy"
elif prob < 0.85:
    state = "very drowsy"
else:
    state = "critical drowsiness"

print(f"Prediction probability: {prob:.4f}")
print(f"Predicted class: {state} (0-0.4 = alert, 0.4-0.7 = slightly drowsy,0.7-0.85 = very drowsy, 0.85-1 = critical drowsiness)")

# label = int(prob > 0.5)

# print(f"Prediction probability: {prob:.4f}")
# print(f"Predicted class: {label} (1 = drowsy, 0 = alert)")


Model expects: 100x100x1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 395ms/step
Prediction probability: 0.7011
Predicted class: very drowsy (0-0.4 = alert, 0.4-0.7 = slightly drowsy,0.7-0.85 = very drowsy, 0.85-1 = critical drowsiness)


In [8]:
import cv2
import dlib
import numpy as np

# Load face detector and landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/classifier/shape_predictor_68_face_landmarks.dat")

# Feature calculation functions (EAR, PUC, MAR, MOE)
def eye_aspect_ratio(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    A = np.linalg.norm(np.array([x[1]-x[5], y[1]-y[5]]))
    B = np.linalg.norm(np.array([x[2]-x[4], y[2]-y[4]]))
    C = np.linalg.norm(np.array([x[0]-x[3], y[0]-y[3]]))
    return (A + B) / (2.0 * C)

def pupil_to_eye_center_distance(eye):
    x = [point.x for point in eye]
    y = [point.y for point in eye]
    return np.linalg.norm(np.array([x[0]-x[3], y[0]-y[3]]))

def mouth_aspect_ratio(mouth):
    x = [point.x for point in mouth]
    y = [point.y for point in mouth]
    A = np.linalg.norm(np.array([x[13]-x[19], y[13]-y[19]]))
    B = np.linalg.norm(np.array([x[14]-x[18], y[14]-y[18]]))
    C = np.linalg.norm(np.array([x[15]-x[17], y[15]-y[17]]))
    return (A + B + C) / (3.0 * np.linalg.norm(np.array([x[12]-x[16], y[12]-y[16]])))

def mouth_to_eye_ratio(eye, mouth):
    ear = eye_aspect_ratio(eye)
    mar = mouth_aspect_ratio(mouth)
    if ear == 0:  # avoid division by zero
        return 0
    return mar / ear

# Function to extract features from a single image
def extract_features_from_image(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    if len(faces) == 0:
        return None  # no face detected

    face = faces[0]  # take the first detected face
    shape = predictor(gray, face)

    ear = eye_aspect_ratio(shape.parts()[36:42])
    puc = pupil_to_eye_center_distance(shape.parts()[36:42])
    mar = mouth_aspect_ratio(shape.parts()[48:68])
    moe = mouth_to_eye_ratio(shape.parts()[36:42], shape.parts()[48:68])

    # Return as numpy array shaped (1,2,2,1) for your CNN
    feature_vector = np.array([[ear, puc], [mar, moe]]).reshape(1, 2, 2, 1).astype(np.float32)
    return feature_vector

In [9]:
img = cv2.imread(img_path)

features = extract_features_from_image(img)
print("Output of extract_features_from_image():")
print("Type:", type(features))
print("Value:", features.shape)

Output of extract_features_from_image():
Type: <class 'numpy.ndarray'>
Value: (1, 2, 2, 1)


In [11]:
# ---- paste this entire block into a notebook cell ----
import cv2
import dlib
import numpy as np
import tensorflow as tf

# ---------------- User paths / model in session ----------------
# Ensure you already have `model` loaded in the kernel (tf.keras.Model).
# If not, load it before calling the functions below.
PREDICTOR_PATH = "/content/drive/MyDrive/classifier/shape_predictor_68_face_landmarks.dat"

# ---------------- dlib detector / predictor ----------------
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(PREDICTOR_PATH)

# ---------------- Feature extractor (returns shape (4,)) ----------------
def eye_aspect_ratio(eye_points):
    coords = np.array([[p.x, p.y] for p in eye_points], dtype=np.float32)
    A = np.linalg.norm(coords[1] - coords[5])
    B = np.linalg.norm(coords[2] - coords[4])
    C = np.linalg.norm(coords[0] - coords[3])
    if C == 0:
        return 0.0
    return float((A + B) / (2.0 * C))

def pupil_to_eye_center_distance(eye_points):
    coords = np.array([[p.x, p.y] for p in eye_points], dtype=np.float32)
    centroid = coords.mean(axis=0)
    corner_midpoint = (coords[0] + coords[3]) / 2.0
    return float(np.linalg.norm(centroid - corner_midpoint))

def mouth_aspect_ratio(mouth_points):
    coords = np.array([[p.x, p.y] for p in mouth_points], dtype=np.float32)
    # indices relative to mouth_points (20 points)
    # choose three vertical distances and horizontal width
    try:
        A = np.linalg.norm(coords[13] - coords[19])
        B = np.linalg.norm(coords[14] - coords[18])
        C = np.linalg.norm(coords[15] - coords[17])
        horizontal = np.linalg.norm(coords[12] - coords[16])
    except Exception:
        # fallback if geometry unexpected
        return 0.0
    if horizontal == 0:
        return 0.0
    return float((A + B + C) / (3.0 * horizontal))

def extract_features_from_image(frame):
    """
    Robust feature extractor for use with finite-difference perturbations.
    Accepts numpy arrays of any numeric dtype (float32/float64/int), clips to 0..255,
    converts to uint8 as required by OpenCV/dlib, and returns np.array([EAR,PUC,MAR,MOE])
    or None if no face detected.
    """
    if frame is None:
        return None

    # Ensure numeric array
    img = np.asarray(frame)

    # If image is in range [0,1], scale up to [0,255] (heuristic)
    if img.dtype.kind == 'f':
        # If floats appear to be in 0..1, scale; otherwise assume 0..255 but still clip
        if img.max() <= 1.0:
            img = (img * 255.0).astype(np.float32)
        # now clip and cast to uint8
        img = np.clip(img, 0.0, 255.0).astype(np.uint8)
    else:
        # integer types: clip to valid 0..255 and cast to uint8
        img = np.clip(img, 0, 255).astype(np.uint8)

    # If single-channel (H,W), keep as is for detector; if 3-channel, assume BGR (OpenCV)
    if img.ndim == 2:
        gray = img
    elif img.ndim == 3 and img.shape[2] == 3:
        # already BGR uint8 as expected
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        # unexpected channel count: try to reduce/expand to 3 channels
        if img.ndim == 3 and img.shape[2] == 4:
            # drop alpha
            img = img[..., :3]
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        elif img.ndim == 3 and img.shape[2] == 1:
            gray = img[..., 0]
        else:
            # fallback: convert to grayscale using mean across channels
            gray = cv2.cvtColor(img.astype(np.uint8), cv2.COLOR_BGR2GRAY)

    # Run detector on uint8 grayscale
    faces = detector(gray)
    if len(faces) == 0:
        return None

    face = faces[0]
    shape = predictor(gray, face)
    parts = list(shape.parts())

    # Safety: ensure we have 68 landmarks
    if len(parts) < 68:
        return None

    # Landmarks slices (dlib 68-point mapping)
    left_eye = parts[36:42]
    right_eye = parts[42:48]
    mouth = parts[48:68]

    # Helper local functions (same as earlier definitions)
    def eye_aspect_ratio_local(eye_points):
        coords = np.array([[p.x, p.y] for p in eye_points], dtype=np.float32)
        A = np.linalg.norm(coords[1] - coords[5])
        B = np.linalg.norm(coords[2] - coords[4])
        C = np.linalg.norm(coords[0] - coords[3])
        if C == 0:
            return 0.0
        return float((A + B) / (2.0 * C))

    def pupil_to_eye_center_distance_local(eye_points):
        coords = np.array([[p.x, p.y] for p in eye_points], dtype=np.float32)
        centroid = coords.mean(axis=0)
        corner_midpoint = (coords[0] + coords[3]) / 2.0
        return float(np.linalg.norm(centroid - corner_midpoint))

    def mouth_aspect_ratio_local(mouth_points):
        coords = np.array([[p.x, p.y] for p in mouth_points], dtype=np.float32)
        try:
            A = np.linalg.norm(coords[13] - coords[19])
            B = np.linalg.norm(coords[14] - coords[18])
            C = np.linalg.norm(coords[15] - coords[17])
            horizontal = np.linalg.norm(coords[12] - coords[16])
        except Exception:
            return 0.0
        if horizontal == 0:
            return 0.0
        return float((A + B + C) / (3.0 * horizontal))

    left_ear = eye_aspect_ratio_local(left_eye)
    right_ear = eye_aspect_ratio_local(right_eye)
    ear = float((left_ear + right_ear) / 2.0)

    left_puc = pupil_to_eye_center_distance_local(left_eye)
    right_puc = pupil_to_eye_center_distance_local(right_eye)
    puc = float((left_puc + right_puc) / 2.0)

    mar = mouth_aspect_ratio_local(mouth)
    moe = float(mar / (ear if ear != 0 else 1e-6))

    return np.array([ear, puc, mar, moe], dtype=np.float32)


# ---------------- Feature importance routine ----------------
def _to_float_img(img):
    # keep original pixel units (0..255) for finite differences
    if img.dtype == np.uint8:
        return img.astype(np.float32)
    return img.astype(np.float32)

def _infer_model_input_shape(model):
    # tries to get (H,W,C) from model.input_shape
    try:
        s = model.input_shape
    except Exception:
        # fallback to first layer
        s = model.layers[0].input_shape
    # Expect formats like (None,H,W,C)
    if isinstance(s, tuple) and len(s) >= 4:
        H = int(s[1]) if s[1] is not None else None
        W = int(s[2]) if s[2] is not None else None
        C = int(s[3]) if s[3] is not None else None
        return (H, W, C)
    # If channels-first (None,C,H,W)
    if isinstance(s, tuple) and len(s) >= 4:
        C = int(s[1]) if s[1] is not None else None
        H = int(s[2]) if s[2] is not None else None
        W = int(s[3]) if s[3] is not None else None
        return (H, W, C)
    raise RuntimeError("Unable to infer model input shape from model.input_shape.")

def _default_preprocess(img, target_hw=None, target_c=None):
    """
    Converts BGR OpenCV image to model input tensor:
    - if model expects 3 channels: convert to RGB
    - if model expects 1 channel: convert to grayscale
    - resizes to target_hw if provided
    - scales to [0,1] (divides by 255.0)
    Returns tf.Tensor shape (1,H,W,C) dtype float32 and scale factor (how model_input = pixel * scale)
    """
    imgf = _to_float_img(img)  # pixel units 0..255
    if target_hw is not None:
        h, w = target_hw
        imgf = cv2.resize(imgf, (w, h))
    cur_c = imgf.shape[2] if imgf.ndim == 3 else 1
    if target_c == 1 and cur_c == 3:
        img_proc = cv2.cvtColor(imgf.astype(np.uint8), cv2.COLOR_BGR2GRAY)
        img_proc = np.expand_dims(img_proc.astype(np.float32), axis=-1)
    elif target_c == 3 and cur_c == 1:
        img_proc = np.repeat(imgf, 3, axis=2)
    elif target_c == 3 and cur_c == 3:
        img_proc = cv2.cvtColor(imgf.astype(np.uint8), cv2.COLOR_BGR2RGB).astype(np.float32)
    else:
        # fallback: keep as-is
        img_proc = imgf
    # scale to [0,1]
    model_input = img_proc.astype(np.float32) / 255.0
    return tf.convert_to_tensor(model_input[None, ...], dtype=tf.float32), (1.0 / 255.0)  # scale = d(model_input)/d(pixel)

def feature_importances_via_patch_fd_v2(model,
                                        orig_img,
                                        extract_features_fn,
                                        preprocess_for_model=None,
                                        patch_size=16,
                                        eps=4.0,
                                        feature_names=['EAR','PUC','MAR','MOE'],
                                        target_class=None):
    """
    model: TF model
    orig_img: original HxWxC BGR image in pixel units (0..255) as numpy array
    extract_features_fn: function(frame)->np.array([4,]) operating on images in same format as orig_img
    preprocess_for_model: optional function(img)->(tensor shape (1,H,W,C), scale) where scale = d(model_input)/d(pixel)
                          If None, default preprocess (resize & /255.0) is used.
    patch_size: patch side in pixels for finite differences
    eps: finite-diff amplitude in pixel units (0..255). e.g., 4.0
    """
    img_pixels = _to_float_img(orig_img)
    H_img, W_img = img_pixels.shape[0], img_pixels.shape[1]

    # Determine model input shape and prepare model_input tensor
    H_model, W_model, C_model = _infer_model_input_shape(model)
    target_hw = (H_model, W_model) if H_model is not None and W_model is not None else None

    if preprocess_for_model is None:
        model_input_tensor, scale = _default_preprocess(img_pixels, target_hw=target_hw, target_c=C_model)
    else:
        # preprocess_for_model should return (tensor, scale) OR just tensor (then we assume scale=1/255)
        res = preprocess_for_model(img_pixels)
        if isinstance(res, tuple) and len(res) == 2:
            model_input_tensor, scale = res
        else:
            model_input_tensor = res
            # assume normalized /255
            scale = (1.0 / 255.0)

    # Ensure model_input_tensor is tf.Variable for gradient computation
    img_tf_var = tf.Variable(model_input_tensor)

    # Compute gradient dY/d(model_input)
    with tf.GradientTape() as tape:
        tape.watch(img_tf_var)
        preds = model(img_tf_var)
        preds = tf.convert_to_tensor(preds)
        # turn preds into scalar
        if preds.shape.rank == 1 or (preds.shape.rank == 2 and preds.shape[1] == 1):
            out_scalar = tf.reshape(preds, [preds.shape[0]])[0]
        else:
            if target_class is not None:
                out_scalar = tf.reshape(preds[:, target_class], [])
            else:
                # use predicted class score
                argmax = tf.argmax(preds, axis=1)[0]
                out_scalar = tf.reshape(preds[0, argmax], [])
    grad_dy_dmodel = tape.gradient(out_scalar, img_tf_var)
    if grad_dy_dmodel is None:
        raise RuntimeError("Gradient is None. Check model and preprocess.")

    # grad_dy_dmodel has same shape as model_input (1,Hm,Wm,Cm)
    grad_dy_dmodel = grad_dy_dmodel.numpy()[0]  # Hm x Wm x Cm

    # Convert gradient to pixel units: dY/d(pixel) = dY/d(model_input) * d(model_input)/d(pixel)
    # scale = d(model_input)/d(pixel) (e.g., 1/255), so multiply
    grad_dy_dpixel = grad_dy_dmodel * scale  # now expresses change in output per 1 pixel unit change

    # Compute base features on original image (pixel-domain)
    base_features = extract_features_fn(img_pixels)
    if base_features is None:
        raise RuntimeError("No face/features detected in the original image.")
    if base_features.shape[0] != len(feature_names):
        raise ValueError("extract_features_fn must return len(feature_names) features.")

    # We'll compute dF/dI (per-pixel per-channel in pixel units) by patch finite differences on original image
    n_patches_h = int(np.ceil(H_img / patch_size))
    n_patches_w = int(np.ceil(W_img / patch_size))

    # dF_dI needs to be in model-input channel-space (Cm). We'll map later.
    # First compute per-pixel per-original-channel derivatives (orig has C_orig channels)
    C_orig = img_pixels.shape[2]
    # We'll compute dF/dI in the original image channels, then map to model channels.
    dF_dI_orig = np.zeros((len(feature_names), H_img, W_img, C_orig), dtype=np.float32)

    for ph in range(n_patches_h):
        for pw in range(n_patches_w):
            y0 = ph * patch_size
            x0 = pw * patch_size
            y1 = min(H_img, y0 + patch_size)
            x1 = min(W_img, x0 + patch_size)
            plus_img = img_pixels.copy()
            minus_img = img_pixels.copy()

            plus_img[y0:y1, x0:x1, :] += eps
            minus_img[y0:y1, x0:x1, :] -= eps

            plus_img = np.clip(plus_img, 0.0, 255.0)
            minus_img = np.clip(minus_img, 0.0, 255.0)

            f_plus = extract_features_fn(plus_img)
            f_minus = extract_features_fn(minus_img)
            if f_plus is None or f_minus is None:
                # skip patch if features can't be computed
                continue
            df_patch = (f_plus - f_minus) / (2.0 * eps)  # shape (4,)
            patch_area = float((y1 - y0) * (x1 - x0))
            if patch_area <= 0:
                continue
            per_pixel_df = (df_patch / patch_area).reshape(len(feature_names), 1, 1)  # per-pixel per-channel scalar (applied equally to channels)

            # assign to each original channel equally (we perturbed all channels at once)
            for i in range(len(feature_names)):
                # assume per-channel effect is same across C_orig when we added same eps to all channels
                dF_dI_orig[i, y0:y1, x0:x1, :] = per_pixel_df[i]

    # Now map dF_dI_orig (orig channels) into model-input channel space (Cm).
    # Strategy:
    #  - If Cm == C_orig: direct mapping by resizing to model-input resolution if needed.
    #  - If model input is resized (Hm,Wm) != original (H_img,W_img): resize each channel map to Hm x Wm.
    #  - If Cm == 1 and C_orig == 3: collapse channels (sum or weighted)
    #  - If Cm == 3 and C_orig == 1: replicate across channels
    Hm = grad_dy_dpixel.shape[0]
    Wm = grad_dy_dpixel.shape[1]
    Cm = grad_dy_dpixel.shape[2]

    dF_dI_modelspace = np.zeros((len(feature_names), Hm, Wm, Cm), dtype=np.float32)

    for i in range(len(feature_names)):
        # for each original channel, resize its HxW map to Hm x Wm
        # then map channels
        # build temp array shape (Hm, Wm, C_orig)
        resized_channels = []
        for ch in range(C_orig):
            channel_map = dF_dI_orig[i, :, :, ch].astype(np.float32)
            # resize: cv2.resize expects (w,h)
            resized = cv2.resize(channel_map, (Wm, Hm), interpolation=cv2.INTER_LINEAR)
            resized_channels.append(resized)
        resized_channels = np.stack(resized_channels, axis=-1)  # Hm x Wm x C_orig

        if Cm == C_orig:
            dF_dI_modelspace[i] = resized_channels
        elif Cm == 1 and C_orig == 3:
            # collapse RGB by summation (we perturbed all channels equally so collapse by mean)
            collapsed = np.mean(resized_channels, axis=-1)
            dF_dI_modelspace[i, :, :, 0] = collapsed
        elif Cm == 3 and C_orig == 1:
            # replicate grayscale to 3-channels
            for ch in range(3):
                dF_dI_modelspace[i, :, :, ch] = resized_channels[:, :, 0]
        else:
            # fallback: if channels differ, distribute equally across Cm
            for ch in range(Cm):
                dF_dI_modelspace[i, :, :, ch] = np.mean(resized_channels, axis=-1) / float(Cm)

    # Finally, compute dY/dF_i = sum_{pixels,channels} dY/d(pixel) * dF/d(pixel)
    importances_raw = {}
    for i, name in enumerate(feature_names):
        dot = float(np.sum(grad_dy_dpixel * dF_dI_modelspace[i]))
        importances_raw[name] = dot

    abs_vals = {k: abs(v) for k, v in importances_raw.items()}
    total = sum(abs_vals.values()) + 1e-12
    normalized = {k: abs_vals[k] / total for k in feature_names}

    return {
        "raw": importances_raw,         # signed sensitivities
        "magnitude": abs_vals,
        "normalized": normalized,
        "base_features": base_features
    }

# ---------------- Example usage ----------------
# Make sure `model` is defined in your notebook (tf.keras.Model).
# Make sure you have an image at IMG_PATH.
img = cv2.imread(img_path)
if img is None:
    raise FileNotFoundError(f"Image not found: {img_path}")

# Run feature importance
result = feature_importances_via_patch_fd_v2(model, img, extract_features_from_image,
                                            preprocess_for_model=None,
                                            patch_size=16, eps=4.0,
                                            feature_names=['EAR','PUC','MAR','MOE'],
                                            target_class=None)

print("Base features:", result['base_features'])
print("Raw dY/dF:", result['raw'])
print("Magnitude:", result['magnitude'])
print("Normalized:", result['normalized'])
print(result)
# --------------------------------------------------


Base features: [0.2252123  0.82143927 0.02066149 0.09174228]
Raw dY/dF: {'EAR': 4.784571387972392e-07, 'PUC': 2.892115844588261e-06, 'MAR': 5.346513987092294e-08, 'MOE': 1.3120636310759437e-07}
Magnitude: {'EAR': 4.784571387972392e-07, 'PUC': 2.892115844588261e-06, 'MAR': 5.346513987092294e-08, 'MOE': 1.3120636310759437e-07}
Normalized: {'EAR': 0.13457780640812, 'PUC': 0.8134785222794995, 'MAR': 0.015038382040279935, 'MOE': 0.03690500799757159}
{'raw': {'EAR': 4.784571387972392e-07, 'PUC': 2.892115844588261e-06, 'MAR': 5.346513987092294e-08, 'MOE': 1.3120636310759437e-07}, 'magnitude': {'EAR': 4.784571387972392e-07, 'PUC': 2.892115844588261e-06, 'MAR': 5.346513987092294e-08, 'MOE': 1.3120636310759437e-07}, 'normalized': {'EAR': 0.13457780640812, 'PUC': 0.8134785222794995, 'MAR': 0.015038382040279935, 'MOE': 0.03690500799757159}, 'base_features': array([0.2252123 , 0.82143927, 0.02066149, 0.09174228], dtype=float32)}


In [13]:
# -------------------------------
# Integrate Mistral explanation (paste after you have `result` and `prob`)
# -------------------------------
import os
import requests
import numpy as np

# --- CONFIG: set your API key (better: pull from env var) ---
API_KEY = os.getenv("MISTRAL_API_KEY", "xGbwkJFTpe7BpsA0iyH462sYW8QPXFNs")  # replace or set env var
MODEL = "mistral-tiny"   # or mistral-small / mistral-medium
API_URL = "https://api.mistral.ai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# --- Safety checks ---
if API_KEY is None or API_KEY == "" or "your_api_key" in API_KEY.lower():
    raise ValueError("Please set API_KEY (put your Mistral key in MISTRAL_API_KEY env var or replace the placeholder).")

# --- Derive prediction label (use prob or pred_class if you have it) ---
# If you have pred_class as a binary label, you can use that. Otherwise use prob threshold.
# try:
#     is_drowsy = None
#     # prefer pred_class if available
#     if 'pred_class' in globals():
#         try:
#             # pred_class may be shape (1,1) or (1,)
#             is_drowsy = bool(int(np.ravel(state)[0]) == 1)
#         except Exception:
#             is_drowsy = None
#     if is_drowsy is None:
#         # fallback to prob (assumes prob in 0..1)
#         is_drowsy = float(prob) > 0.4
# except Exception:
#     # final fallback
#     is_drowsy = False

# pred_label = "Drowsy" if is_drowsy else "Alert"

# is_drowsy = stage != "alert"

# --- Build importance dict from `result` ---
# Prefer using absolute magnitudes to explain which regions contributed most
if 'result' not in globals():
    raise RuntimeError("Missing `result` variable. Run the feature importance routine first.")

# Use magnitude if available, else use absolute of raw
if 'magnitude' in result:
    importance_map = result['magnitude']
else:
    importance_map = {k: abs(v) for k, v in result.get('raw', {}).items()}

# Keep ordering consistent
feature_order = ['EAR', 'PUC', 'MAR', 'MOE']
vals = np.array([importance_map.get(k, 0.0) for k in feature_order], dtype=float)

# Normalize (for scaling in prompt only)
vals_norm = vals / (vals.max() + 1e-12)

# Prepare short textual summary (numbers included only for internal clarity; the assistant is instructed not to read them aloud)
importance_text = "\n".join([f"{k}: {float(importance_map.get(k,0.0)):.6e}" for k in feature_order])

# -------------------------------
# Build Mistral prompt
# -------------------------------
prompt = f"""
You are an empathetic AI driving assistant that explains driver alertness.

Prediction: {state}
Prediction Probability : {prob}

Feature importance values (absolute, gradient-based):
{importance_text}
These values are from the drowsiness classifier explaining how much weight each facial cue gave for the classification.
the classifer works like:
0.0 - 0.4 : alert(means not drowsy)
0.41 - 0.7 : slightly drowsy
0.71 - 0.85 : very drowsy
0.86 - 1.00 : critical drowsiness

mention the state of the driver first through finding the stage by putting the prediction probability into the classifier values, remember if the value is near the border for example 0.3956 it is still alert and not drowsy, if the valeu is less than 0.4 say the driver is not drowsy and alert

if alert stop here else if drowsy, Explain to the driver what this means in a friendly and understanding way.
If they are drowsy, warn them depending on the stage of drowsiness decided by the if else logic and the facial regions the system found indicative of drowsiness aka the regions with high values
depending on the stage change the meesage for exmaple if crtitcal make the message more urgent. Incase the system is making a false prediction tell him to press the cancel button
Keep it short, human, and supportive.
"""

# -------------------------------
# Query function
# -------------------------------
def mistral_chat(prompt, model=MODEL, api_url=API_URL, headers=HEADERS, timeout=15):
    data = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }
    try:
        resp = requests.post(api_url, headers=headers, json=data, timeout=timeout)
    except Exception as e:
        print("❌ Request error:", str(e))
        return None

    if resp.status_code != 200:
        print("❌ API Error", resp.status_code, resp.text)
        return None
    try:
        payload = resp.json()
        # Defensive: navigate response shape
        content = payload["choices"][0]["message"]["content"]
        return content
    except Exception as e:
        print("❌ Response parsing error:", str(e))
        print("Raw response:", resp.text[:1000])
        return None

# -------------------------------
# Call Mistral and print result
# -------------------------------
assistant_text = mistral_chat(prompt)
if assistant_text is None:
    print("No assistant response received.")
else:
    print("🤖 Driving assistant message:\n")
    print(assistant_text)


🤖 Driving assistant message:

Based on the prediction, you seem to be in a state of high drowsiness, with a probability of 0.7011. This means you might be feeling sleepy and less alert than usual. Don't worry, it happens to us all! However, it's important to stay focused while driving. Let's break down the facial cues that are contributing to this classification:

* EAR (Eye Aspect Ratio) - 4.78e-07 - This suggests your eyes may be closing slightly more than usual, which is a common sign of drowsiness.
* PUC (Periocular Closure) - 2.89e-06 - This could mean your eyelids are drooping slightly, another indicator of drowsiness.
* MAR (Mouth Aspect Ratio) - 5.35e-08 - This feature seems to have little impact on the classification, suggesting your mouth may not be a significant factor in your current drowsiness.
* MOE (Magnitude of Excursion) - 1.31e-07 - This feature measures the movement of your eyes, and a low value could mean your eyes are moving less than usual, which can be a sign of 